In [10]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'/home/joaquin/Documents/GitHub/skforecast'

In [11]:
import platform
import psutil
import skforecast
import pandas as pd
import numpy as np
import scipy
import sklearn
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMRegressor
from skforecast.recursive import ForecasterRecursiveMultiSeries
from skforecast.model_selection import grid_search_forecaster_multiseries
from skforecast.utils.utils import align_series_and_exog_multiseries
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import make_column_transformer
from skforecast.preprocessing import series_long_to_dict
from skforecast.preprocessing import exog_long_to_dict
from skforecast.datasets import fetch_dataset
from skforecast.utils import check_preprocess_series


%load_ext pyinstrument
%load_ext line_profiler

The pyinstrument extension is already loaded. To reload it, use:
  %reload_ext pyinstrument
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [16]:
import pandas as pd

n = 5000
series = pd.DataFrame(
    {f"series_{i}": np.random.randint(1, 100, n) for i in range(1, 100)
    },
    index=pd.date_range(start="2023-01-01", periods=n, freq="D"),
)

freq = series.index.freq
series.index.name = "datetime"
series = series.reset_index()
series = pd.melt(series, id_vars="datetime", var_name="series_id", value_name="value")
series = series.groupby("series_id").apply(
    lambda x: x.set_index("datetime").asfreq("D"), include_groups=False
)
display(series)

exog = pd.DataFrame(
    {
        "exog_1": np.random.randint(1, 10, n),
        "exog_2": np.random.randint(1, 10, n),
    },
    index=pd.date_range(start="2023-01-01", periods=n, freq="D"),
)
exog.index.name= "datetime"
exog = [exog.assign(series_id=f"series_{i}") for i in range(1, 100)]
# exog = [exog_i.sample(frac=0.8) for exog_i in exog]
exog = pd.concat(exog)
exog = exog.set_index(["series_id", exog.index])
display(exog)

# Formato dict
series_dict = {
    sid: series.loc[sid]['value'] for sid in series.index.levels[0]
}

exog_dict = {
    sid: exog.loc[sid]for sid in exog.index.levels[0]
}


value
series_id datetime         
series_1  2023-01-01     30
          2023-01-02     83
          2023-01-03     45
          2023-01-04     20
          2023-01-05     47
...                     ...
series_99 2036-09-04     87
          2036-09-05     24
          2036-09-06     83
          2036-09-07     60
          2036-09-08     94

[495000 rows x 1 columns]

exog_1  exog_2
series_id datetime                  
series_1  2023-01-01       8       3
          2023-01-02       6       2
          2023-01-03       9       3
          2023-01-04       8       9
          2023-01-05       9       8
...                      ...     ...
series_99 2036-09-04       3       2
          2036-09-05       3       3
          2036-09-06       6       2
          2036-09-07       1       7
          2036-09-08       4       8

[495000 rows x 2 columns]

In [13]:
forecaster = ForecasterRecursiveMultiSeries(regressor=LGBMRegressor(), lags=5, transformer_exog=StandardScaler())
forecaster.transformer_series_ = {k: StandardScaler() for k in series.index.get_level_values(0).unique()}
forecaster.differentiator_ = {k: None for k in series.index.get_level_values(0).unique()}

In [14]:
# %%timeit -r 5 -n 5
# _ = forecaster._create_train_X_y(
#     series = series,
#     exog=exog
# )

In [15]:
%%timeit -r 5 -n 5
_ = forecaster._create_train_X_y_v2(
    series = series,
    exog=exog
)

╭──────────────────────────────── MissingValuesWarning ────────────────────────────────╮
│ NaNs detected in `X_train`. Some regressors do not allow NaN values during training. │
│ If you want to drop them, set `forecaster.dropna_from_series = True`.                │
│                                                                                      │
│ Category : MissingValuesWarning                                                      │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/recursive/_forecaster_recursive │
│ _multiseries.py:1524                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=MissingValuesWarning)            │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────── MissingValuesWarning ────────────────────────────────╮
│ NaNs detected in `X_train`. Some regressors do not allow NaN values during training. │
│ If you want to drop them, set `forecaster.dropna_from_series = True`.                │
│                                                                                      │
│ Category : MissingValuesWarning                                                      │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/recursive/_forecaster_recursive │
│ _multiseries.py:1524                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=MissingValuesWarning)            │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────── MissingValuesWarning ────────────────────────────────╮
│ NaNs detected in `X_train`. Some regressors do not allow NaN values during training. │
│ If you want to drop them, set `forecaster.dropna_from_series = True`.                │
│                                                                                      │
│ Category : MissingValuesWarning                                                      │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/recursive/_forecaster_recursive │
│ _multiseries.py:1524                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=MissingValuesWarning)            │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────── MissingValuesWarning ────────────────────────────────╮
│ NaNs detected in `X_train`. Some regressors do not allow NaN values during training. │
│ If you want to drop them, set `forecaster.dropna_from_series = True`.                │
│                                                                                      │
│ Category : MissingValuesWarning                                                      │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/recursive/_forecaster_recursive │
│ _multiseries.py:1524                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=MissingValuesWarning)            │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────── MissingValuesWarning ────────────────────────────────╮
│ NaNs detected in `X_train`. Some regressors do not allow NaN values during training. │
│ If you want to drop them, set `forecaster.dropna_from_series = True`.                │
│                                                                                      │
│ Category : MissingValuesWarning                                                      │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/recursive/_forecaster_recursive │
│ _multiseries.py:1524                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=MissingValuesWarning)            │
╰──────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────── MissingValuesWarning ────────────────────────────────╮
│ NaNs detected in `X_train`. Some regressors do not allow NaN values during training. │
│ If you want to drop them, set `forecaster.dropna_from_series = True`.                │
│                                                                                      │
│ Category : MissingValuesWarning                                                      │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/recursive/_forecaster_recursive │
│ _multiseries.py:1524                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=MissingValuesWarning)            │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────── MissingValuesWarning ────────────────────────────────╮
│ NaNs detected in `X_train`. Some regressors do not allow NaN values during training. │
│ If you want to drop them, set `forecaster.dropna_from_series = True`.                │
│                                                                                      │
│ Category : MissingValuesWarning                                                      │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/recursive/_forecaster_recursive │
│ _multiseries.py:1524                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=MissingValuesWarning)            │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────── MissingValuesWarning ────────────────────────────────╮
│ NaNs detected in `X_train`. Some regressors do not allow NaN values during training. │
│ If you want to drop them, set `forecaster.dropna_from_series = True`.                │
│                                                                                      │
│ Category : MissingValuesWarning                                                      │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/recursive/_forecaster_recursive │
│ _multiseries.py:1524                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=MissingValuesWarning)            │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────── MissingValuesWarning ────────────────────────────────╮
│ NaNs detected in `X_train`. Some regressors do not allow NaN values during training. │
│ If you want to drop them, set `forecaster.dropna_from_series = True`.                │
│                                                                                      │
│ Category : MissingValuesWarning                                                      │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/recursive/_forecaster_recursive │
│ _multiseries.py:1524                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=MissingValuesWarning)            │
╰──────────────────────────────────────────────────────────────────────────────────────╯

1.15 s ± 94.1 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [7]:
def funt_to_profile(forecaster, series, exog):
    _ = forecaster._create_train_X_y_v2(
                            series = series,
                            exog=exog
                        )

%lprun -f forecaster._create_train_X_y_v2 funt_to_profile(forecaster, series, exog)

Timer unit: 1e-09 s

Total time: 1.47957 s
File: /home/joaquin/Documents/GitHub/skforecast/skforecast/recursive/_forecaster_recursive_multiseries.py
Function: _create_train_X_y_v2 at line 1272

Line #      Hits         Time  Per Hit   % Time  Line Contents
  1272                                               def _create_train_X_y_v2(
  1273                                                   self,
  1274                                                   series: pd.DataFrame,
  1275                                                   exog: pd.Series | pd.DataFrame | None = None,
  1276                                                   store_last_window: bool | list[str] = True,
  1277                                               ) -> tuple[
  1278                                                   pd.DataFrame,
  1279                                                   pd.Series,
  1280                                                   dict[str, pd.Index],
  1281                              

In [4]:
%%timeit -r 5 -n 5
_ = forecaster._create_train_X_y(
    series = series_dict,
    exog=exog_dict
)

428 ms ± 44.8 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [5]:
def funt_to_profile(forecaster, series_dict, exog_dict):
    _ = forecaster._create_train_X_y(
                            series = series_dict,
                            exog=exog_dict
                        )

%lprun -f forecaster._create_train_X_y funt_to_profile(forecaster, series_dict, exog_dict)

Timer unit: 1e-09 s

Total time: 0.414201 s
File: /home/joaquin/miniconda3/envs/skforecast_16_py12/lib/python3.12/site-packages/skforecast/recursive/_forecaster_recursive_multiseries.py
Function: _create_train_X_y at line 960

Line #      Hits         Time  Per Hit   % Time  Line Contents
   960                                               def _create_train_X_y(
   961                                                   self,
   962                                                   series: pd.DataFrame | dict[str, pd.Series | pd.DataFrame],
   963                                                   exog: pd.Series | pd.DataFrame | dict[str, pd.Series | pd.DataFrame] | None = None,
   964                                                   store_last_window: bool | list[str] = True,
   965                                               ) -> tuple[
   966                                                   pd.DataFrame,
   967                                                   pd.Series,
   968  

In [8]:
def funt_to_profile(series):
    check_preprocess_series(series=series)

%lprun -f check_preprocess_series funt_to_profile(series)

Timer unit: 1e-09 s

Total time: 0.00257965 s
File: /home/joaquin/miniconda3/envs/skforecast_16_py12/lib/python3.12/site-packages/skforecast/utils/utils.py
Function: check_preprocess_series at line 2448

Line #      Hits         Time  Per Hit   % Time  Line Contents
  2448                                           def check_preprocess_series(
  2449                                               series: pd.DataFrame | dict[str, pd.Series | pd.DataFrame],
  2450                                           ) -> tuple[dict[str, pd.Series], dict[str, pd.Index]]:
  2451                                               """
  2452                                               Check and preprocess `series` argument in `ForecasterRecursiveMultiSeries` class.
  2453                                           
  2454                                               - If `series` is a pandas DataFrame, it is converted to a dict of pandas 
  2455                                               Series and index

In [9]:
def funt_to_profile(series_dict):
    check_preprocess_series(series=series_dict)

%lprun -f check_preprocess_series funt_to_profile(series_dict)

Timer unit: 1e-09 s

Total time: 0.0332219 s
File: /home/joaquin/miniconda3/envs/skforecast_16_py12/lib/python3.12/site-packages/skforecast/utils/utils.py
Function: check_preprocess_series at line 2448

Line #      Hits         Time  Per Hit   % Time  Line Contents
  2448                                           def check_preprocess_series(
  2449                                               series: pd.DataFrame | dict[str, pd.Series | pd.DataFrame],
  2450                                           ) -> tuple[dict[str, pd.Series], dict[str, pd.Index]]:
  2451                                               """
  2452                                               Check and preprocess `series` argument in `ForecasterRecursiveMultiSeries` class.
  2453                                           
  2454                                               - If `series` is a pandas DataFrame, it is converted to a dict of pandas 
  2455                                               Series and index 

In [26]:
# %%pyinstrument
# x_train, y_train = forecaster.create_train_X_y(
#     series = series_dict,
#     exog=exog_dict
# )

In [38]:
import numpy as np
import pandas as pd
from numba import njit
from collections import defaultdict


n = 5000
series = pd.DataFrame(
    {i: np.random.randint(1, 100, n) for i in range(1, 100)
    },
    index=pd.date_range(start="2023-01-01", periods=n, freq="h"),
)

freq = series.index.freq
series.index.name = "datetime"
series = series.reset_index()
series = pd.melt(series, id_vars="datetime", var_name="series_id", value_name="value")
series = series.groupby("series_id").apply(
    lambda x: x.set_index("datetime").asfreq("D"), include_groups=False
)

In [39]:


# Extract MultiIndex levels
ids = series.index.get_level_values(0).to_numpy()
dates = series.index.get_level_values(1).astype('datetime64[ns]').to_numpy()

# Ensure sorting by (id, datetime)
sort_idx = np.lexsort((dates, ids))
ids = ids[sort_idx]
dates_ns = dates[sort_idx].view('int64')

@njit
def most_common_deltas(ids, dates_ns, max_check=1000):
    result_ids = []
    result_deltas = []

    start = 0
    n = len(ids)

    while start < n:
        current_id = ids[start]
        end = start + 1
        while end < n and ids[end] == current_id:
            end += 1

        # Extract time series slice
        series = dates_ns[start:end]
        count = min(len(series) - 1, max_check)
        
        if count < 1:
            result_ids.append(current_id)
            result_deltas.append(-1)  # No data
            start = end
            continue

        delta_counts = {}
        for i in range(count):
            delta = series[i + 1] - series[i]
            if delta in delta_counts:
                delta_counts[delta] += 1
            else:
                delta_counts[delta] = 1

        # Find most common delta
        max_delta = -1
        max_count = -1
        for delta, cnt in delta_counts.items():
            if cnt > max_count:
                max_delta = delta
                max_count = cnt

        result_ids.append(current_id)
        result_deltas.append(max_delta)
        start = end

    return result_ids, result_deltas



In [37]:

unique_ids, deltas_ns = most_common_deltas(ids, dates_ns)

# Convert to pandas timedelta64
freqs = {}
for uid, delta in zip(unique_ids, deltas_ns):
    if delta == -1:
        freqs[uid] = None  # Not enough data
    else:
        freqs[uid] = pd.Timedelta(delta, unit='ns')  # Convert to pandas Timedelta
freqs


{1: Timedelta('1 days 00:00:00'),
 2: Timedelta('1 days 00:00:00'),
 3: Timedelta('1 days 00:00:00'),
 4: Timedelta('1 days 00:00:00'),
 5: Timedelta('1 days 00:00:00'),
 6: Timedelta('1 days 00:00:00'),
 7: Timedelta('1 days 00:00:00'),
 8: Timedelta('1 days 00:00:00'),
 9: Timedelta('1 days 00:00:00'),
 10: Timedelta('1 days 00:00:00'),
 11: Timedelta('1 days 00:00:00'),
 12: Timedelta('1 days 00:00:00'),
 13: Timedelta('1 days 00:00:00'),
 14: Timedelta('1 days 00:00:00'),
 15: Timedelta('1 days 00:00:00'),
 16: Timedelta('1 days 00:00:00'),
 17: Timedelta('1 days 00:00:00'),
 18: Timedelta('1 days 00:00:00'),
 19: Timedelta('1 days 00:00:00'),
 20: Timedelta('1 days 00:00:00'),
 21: Timedelta('1 days 00:00:00'),
 22: Timedelta('1 days 00:00:00'),
 23: Timedelta('1 days 00:00:00'),
 24: Timedelta('1 days 00:00:00'),
 25: Timedelta('1 days 00:00:00'),
 26: Timedelta('1 days 00:00:00'),
 27: Timedelta('1 days 00:00:00'),
 28: Timedelta('1 days 00:00:00'),
 29: Timedelta('1 days 00:00: